In [21]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import time
import copy
import random
#from sklearn.decomposition import PCA
#from sklearn.preprocessing import StandardScale

In [24]:
# define Euclidian distance
def dist(a, b):
    return np.linalg.norm(a - b, None)

In [4]:
# define tau distance between point a and centroid m
# tau variate among different dimensions and different groups
def dist_fun_vtau(a,m,tau):
    dist = np.zeros(len(m))
    for i in range(len(m)):
        d = a - m[i]
        ele = 0
        for j in range(len(d)):
            col = d[j]
            ad = (1-tau[i,j])* sum(col[col<0]**2) + tau[i,j]* sum(col[col>=0]**2)
            ele = ele + ad
        dist[i] = ele
    return dist

In [ ]:
# define tau distance between point a and centroid m
# uniform tau among groups, tau can be the same or different among dimensions
def dist_fun_utau(a,m,tau):
    dist = np.zeros(len(m))
    for i in range(len(m)):
        d = a - m[i]
        ele = 0
        for j in range(len(tau)):
            col = d[j]
            ad = (1-tau[j])* sum(col[col<0]**2) + tau[j]* sum(col[col>=0]**2)
            ele = ele + ad
        dist[i] = ele
    return dist

In [ ]:
# calculate expectile of a group
def expectile_fun(group, tau):
    e = np.mean(group, axis=0)
    e_new = np.zeros(e.shape)
    while dist(e_new , e) != 0:
        c = group[:,:]- e
        e = copy.deepcopy(e_new)
        for i in range(len(c[0])):
            d = c[:,i]
            a_co = group[:,i]
            neg = a_co[d<0]
            pos = a_co[d>=0]
            norm = tau[i]*len(pos)+ (1-tau[i])*len(neg)
            e_new[i] = (tau[i]* sum(pos) + (1-tau[i])* sum(neg))/norm
    return  e_new

In [ ]:
# calculate tau
def tau_fun(points, mu):
    tau_list = []
    dis = points - mu
    for i in range(len(mu)):
        res = dis[:,i]
        e_neg = -sum(res[res < 0])/len(res[res < 0])
        e_pos = sum(res[res >= 0])/len(res[res >= 0])
        c = e_neg/e_pos
        tau = c/(1+c)
        tau_list.append(tau)
    return tau_list

In [ ]:
def tau_fun(group, e):
    res = group - e
    neg = np.array([res<0],dtype = np.int32)
    pos = np.array([res>=0],dtype= np.int32)
    neg_s = -(neg*res).sum(axis=1)/neg.sum(axis=1)
    pos_s = (pos*res).sum(axis=1)/pos.sum(axis=1)
    c = neg_s/pos_s
    tau = c/(1+c)
    return tau

In [ ]:
# K expectile clustering with pre-specified tau vector
def k_expectile_utau(X, k, tau): 
    X = np.array(X)
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    C = kmeans.cluster_centers_

# To store the value of centroids when it updates
    C_old = np.zeros(C.shape)
# Cluster Lables(0, 1, 2)
    clusters = np.zeros(len(X))
# Error func. - Distance between new centroids and old centroids
    error = dist(C, C_old)
# Loop will run till the error becomes zero
    while error != 0:
    # Assigning each value to its closest cluster
        for i in range(len(X)):
            distances = dist_fun_utau(X[i], C, tau)
            cluster = np.argmin(distances)
            clusters[i] = cluster
    # Storing the old centroid values
        C_old = copy.deepcopy(C)
    # Finding the new centroids by taking the average value
        for d in range(k):
            points = [X[i] for i in range(len(X)) if clusters[i] == d]
            points = np.array(points)
            C[d] = expectile_fun(points,tau)
        error = dist(C, C_old)
    return C, clusters

In [5]:
# K expectile clustering with unknown taus
def k_expectile_vtau(X, k): 
    X = np.array(X)
# Initialize cluster centers as K means cluster centers
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    C = kmeans.cluster_centers_

# To store the value of centroids when it updates
    C_old = np.zeros(C.shape)
    clusters = np.zeros(len(X))
# Initialize tau = 0.5
    tau_list = np.ones((k, len(C[0])))*0.5
# Error func. - Distance between new centroids and old centroids
    error = dist(C, C_old)
# Loop will run till the error 
    while error >= 0.05:
    # Assigning each value to its closest cluster
        for i in range(len(X)):
            for j in range (len(C[0])):
                distances = dist_fun_vtau(X[i], C, tau_list)
                cluster = np.argmin(distances)
                clusters[i] = cluster
    # Storing the old centroid values
        C_old = copy.deepcopy(C)
    # Finding the new centroids and tau
        for d in range(k):
            points = [X[i] for i in range(len(X)) if clusters[i] == d]
            points = np.array(points)
    # Updating taus
            tau = tau_fun(points,C[d])
            C[d] = expectile_fun(points,tau)
            tau_list[d] = tau
        error = dist(C, C_old)
        print(tau_list)
        print(error)
    return C, clusters

In [41]:
#Euclidean distance
def dist(a, b):
    return np.linalg.norm(a - b, None)
#Tau-distance
def tau_dist_fun(x, centroid, tau):
    arr = x - centroid
    tau_ar = tau[None:,:]
    neg = np.array([arr<0],dtype=np.int32)
    pos = np.array([arr>=0],dtype=np.int32)
    w_pos = pos * tau_ar
    w_neg = neg * (1-tau_ar)
    dist = (arr ** 2 * w_pos).sum(axis = arr.ndim) + (arr ** 2 * w_neg).sum(axis = arr.ndim)
    return dist
#get centroids
def get_closest_centroid(x, centroid, tau):
    # Loop over each centroid and compute the distance from data point.
    dist = tau_dist_fun(x, centroid, tau)
    # Get the index of the centroid with the smallest distance to the data point
    assigned_centroids = np.argmin(dist, axis = 2)
    clusters = np.squeeze(assigned_centroids)
    return clusters
#Expectile estimation
def expectile_fun_c(group, tau):
    e = np.mean(group, axis=0)
    e_new = np.zeros(e.shape)
    while dist(e_new , e) != 0:
        res = group - e
        e = copy.deepcopy(e_new)
        neg = np.array([res<0],dtype = np.int32)
        pos = np.array([res>=0],dtype= np.int32)
        norm = pos.sum(axis=1)*tau + neg.sum(axis=1)*(1-tau)
        e_arr = (tau * (group * pos).sum(axis=1) + (1-tau) * (group * neg).sum(axis=1))/norm
        e_new = np.squeeze(e_arr)
    return e_new
#Estimate optimal taus
def tau_fun_c(group, e):
    res = group - e
    neg = np.array([res<0],dtype = np.int32)
    pos = np.array([res>=0],dtype= np.int32)
    neg_s = -(neg*res).sum(axis=1)/neg.sum(axis=1)
    pos_s = (pos*res).sum(axis=1)/pos.sum(axis=1)
    c = neg_s/pos_s
    tau = c/(1+c)
    return tau
# Define K expectile clustering 
# Main function
def k_expectile_vtau_c(X, k):
    X = np.array(X)
# Initialize cluster centers as K means cluster centers
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    C = kmeans.cluster_centers_

# To store the value of centroids when it updates
    C_old = np.zeros(C.shape)
# Initialize tau = 0.5
    tau = np.ones(C.shape)*0.5
# Error func. - Distance between new centroids and old centroids
    error = dist(C, C_old)
# main loop   
    for r in range(30):
        # Get closest centroids to each data point
        assigned_clusters = get_closest_centroid(X[:, None, :], C[None,:, :], tau)
        # Storing the old centroid values
        C_old = copy.deepcopy(C)
        # Compute new centroids
        for c in range(k):
            # Get data points belonging to each cluster 
            cluster_members = X[assigned_clusters == c]
        
            # Compute the centroids of the clusters
            C[c] = expectile_fun_c(cluster_members, tau[c])
        
            # Update the tau
            tau[c] = tau_fun_c(cluster_members, C[c])
        error = dist(C, C_old)
        print(tau)
        print(error)
    return C, assigned_clusters

In [22]:
data_size = 1000
num_iters = 50
num_clusters = 4


# sample from Gaussians 
data1 = np.random.normal((5,5,5), (4, 4, 4), (data_size,3))
data2 = np.random.normal((4,20,20), (3,3,3), (data_size, 3))
data3 = np.random.normal((25, 20, 5), (5, 5, 5), (data_size,3))
data4 = np.random.normal((30, 30, 30), (5, 5, 5), (data_size,3))

# Combine the data to create the final dataset
data = np.concatenate((data1,data2, data3, data4), axis = 0)

np.random.shuffle(data)

# Set random seed for reproducibility 
random.seed(0)

In [38]:
C_e, clusters_e = k_expectile_vtau_c(data, 4)

[[0.49748238 0.49748238 0.50654582]
 [0.49598394 0.50803213 0.51104418]
 [0.475      0.518      0.515     ]
 [0.48664688 0.49950544 0.50346192]]
3.9859325923993215e-14
